## 复习手写MHA
2025-10-08 00:52:55 Wednesday

- torch.max(x,dim=-1,keepdim=True) 中dim=-1的理解？
比如x为2行3列，那么结果应该该是2*1,因为dim=-1，表示计算列中的最大值，即3列中的最大值。


- 如何在一个维度把x分开？
使用 _.chunk()

In [20]:
x=torch.randn(2,3,4)
x_max=torch.max(x,dim=-1).values
ic(x_max.shape)

ic| x_max.shape: torch.Size([2, 3])


torch.Size([2, 3])

In [ ]:
import torch
import torch.nn as nn 
from icecream import ic
def softmax(x):
    x_max = torch.max(x,dim=-1,keepdim=True).values
    ic(x_max.shape)
    exp_x = torch.exp(x-x_max)
    return exp_x/torch.sum(exp_x,dim=-1,keepdim=True)

class mla(nn.Module):
    def __init__(self,hidden_dim,head):
        super().__init__()
        self.hidden_dim=hidden_dim
        self.head = head
        self.qkv_proj= nn.Linear(hidden_dim,hidden_dim*3)
    
    def forward(self,x):
        b,s,d=x.shape
        h=self.head
        d1=d//h
        
        assert d%self.head ==0
        qkv=self.qkv_proj(x)
        q,k,v=qkv.chunk(3,dim=-1)
        
        q= q.reshape(b,s,h,d1).transpose(1,2)
        k= k.reshape(b,s,h,d1).transpose(1,2)
        v= v.reshape(b,s,h,d1).transpose(1,2)

        qk=torch.matmul(q,k.transpose(-1,-2))/(d1**0.5)
        
        ic(qk.shape)
        qk=softmax(qk)

        attention= torch.matmul(qk,v)
        return attention.transpose(1,2).reshape(b,s,d)

x=torch.rand(3,5,10)
mla0=mla(hidden_dim=10,head=2)
x0=mla0(x)
ic(x0.shape)
        




ic| qk.shape: torch.Size([3, 2, 5, 5])
ic| x_max.shape: torch.Size([3, 2, 5, 1])
ic| x0.shape: torch.Size([3, 5, 10])


torch.Size([3, 5, 10])